# Homework
Your homework is to create a new notebook modeled after this one, where:
- You've put all of your cleaning and prepration steps into a functions you load and use in a module.
- You create and train a simple appropriate model for your task.
- You assess the performance using the appropriate metric for your task.
- Compare the performance with your previous best non Deep Neural Network model. This probably will mean you copy and paste your training from your other notebook into this notebook.

In [34]:
import processing_module as pm # module that contains data preprocessing and cleaning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
train = pd.read_csv('/mnt/c/users/efazh/DataProblems/Datasets/Tabular/Zillow/train_2016_v2.csv', )
props = pd.read_csv('/mnt/c/users/efazh/DataProblems/Datasets/Tabular/Zillow/property_2016.csv')

print('Shape of Training Set (2016): ',train.shape)
print('Shape of Properties (2016): ',props.shape)

Shape of Training Set (2016):  (90275, 3)
Shape of Properties (2016):  (90274, 58)


In [15]:
from sklearn.preprocessing import LabelEncoder

for c in props.columns: # Converts all categorical values to numerical values via Label Encoding
    props[c]=props[c].fillna(-1)
    if props[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(props[c].values))
        props[c] = lbl.transform(list(props[c].values))

train_df = train.merge(props, how='left', on='parcelid') #Merged the old dataset with the converted properties set

In [17]:
X_train = train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
X_test = props.drop(['parcelid'], axis=1)
# printing out the shape        
print('Shape of train dataset: {}\nShape of test dataset: {}'.format(X_train.shape, X_test.shape))

# taking out potential outliers to fit our model better
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.42 ]
X_train=train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)

Shape of train dataset: (90275, 57)
Shape of test dataset: (90274, 57)


# Sequential

In [19]:
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [21]:
X_train.shape[1]

57

In [23]:
# 1st Hidden layer
X_train.shape[1]*12 + 12

696

In [24]:
# 2nd Hidden layer
12 * 8 + 8

104

In [25]:
y_train.to_numpy().astype("float")

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [29]:
from keras.activations import relu
import keras

in_x = Input(shape = X_train.shape[1:])
x = keras.layers.Normalization(axis=-1)(in_x)
x = Dense(12, input_dim=X_train.shape[1], activation='relu')(in_x)
x = Dense(8)(x)
x = relu(x)
out_x = Dense(1, activation='sigmoid')(x)

model = Model(in_x,out_x)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 57)]              0         
                                                                 
 dense_8 (Dense)             (None, 12)                696       
                                                                 
 dense_9 (Dense)             (None, 8)                 104       
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 8)                 0         
                                                                 
 dense_10 (Dense)            (None, 1)                 9         
                                                                 
Total params: 809
Trainable params: 809
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
history=model.fit(X_train.to_numpy().astype("float"), 
                  y_train.to_numpy().astype("float"), 
                  validation_data=(X_test.to_numpy(),y_test.to_numpy()), 
                  epochs=100, batch_size=10)

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [ ]:
print (history.history)

In [ ]:
plt.plot(range(len(history.history["loss"])),history.history["loss"],label="Training Loss")
plt.plot(range(len(history.history["val_loss"])),history.history["val_loss"],label="Validation Loss")
plt.legend()

In [ ]:
plt.plot(range(len(history.history["accuracy"])),history.history["accuracy"],label="Training accuracy")
plt.plot(range(len(history.history["val_accuracy"])),history.history["val_accuracy"],label="Validation accuracy")
plt.legend()

# Evaluating Model

In [ ]:
scores = model.evaluate(X_test, y_test)
print(scores)

In [ ]:
model.predict(X_test)

# Confirm Distribution Visually

In [ ]:
X_Test_0 = X_Test[(y_Test==0).to_numpy()]
X_Test_1 = X_Test[(y_Test==1).to_numpy()]

In [ ]:
hist,bins,_= plt.hist(model.predict(X_Test_0),density=1,bins=50,alpha=0.5,label="Target 0")
hist,bins,_= plt.hist(model.predict(X_Test_1),density=1,bins=bins,alpha=0.5,label="Target 1")
plt.legend()